# Code for practical part of PDC Project

Hard limits to respect :
- Codewords have to be in the space [-3/2, 3/2] x [-3/2, 3/2] -> **limits the energy per symbol**
- The average energy of the vector **X** should be <= 1 -> **also limits the energy per symbol** 
- The dimension of the encoded vector **X** should not exceed n <= 100 -> **limits the number of symbols we can send**

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

## Utility functions to help code the encoder/decoder

In [2]:
# Computes the value of the Q-Function
def q_function(x):
    return (1/2) * sp.erfc(x/np.sqrt(2))

# Gets every possible codeword lengths for which the length divides the total length of the original bitstring
def compute_codeword_lengths(bit_str):
    n = len(bit_str)
    return [i for i in range(1, n+1) if n % i == 0]

# Gets the whole alphabet from a codeword length
def get_alphabet_from_codeword_length(length):
    return [f'{i:0{length}b}' for i in range(0, 2**length)]

# Performs a conversion from ASCII string to binary symbols
def ascii_str_to_binary(ascii_str):
    return str(''.join(format(ord(i), '08b')[1:] for i in ascii_str))

# Performs the inverse conversion, from a binary stream to ASCII characters
def binary_to_ascii_str(binstr):
    byte_binstr = "".join([f"0{byte}" for byte in split_bit_str(binstr, 7)])
    binary_int = int(byte_binstr, 2)
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    return binary_array.decode()

# Splits a bit string into chunks of size 'chunk_size'.
def split_bit_str(bit_str, chunk_size):
    return [bit_str[i:i+chunk_size] for i in range(0, len(bit_str), chunk_size)]

# Gets all the positions where bit_str1 is different from bit_str2 (bit_str1 and bit_str2 MUST HAVE THE SAME SIZE !)
def get_diff_positions(bit_str1, bit_str2):
    return [i for i in range(0, len(bit_str1)) if bit_str1[i] != bit_str2[i]]

# Computes the distance between a symbol and every point of a given constellation and returns the index of the nearest point in the constellation
def minimum_distance_criterion(constellation, symbol):
    return np.argmin([((symbol.real - point.real)**2 + (symbol.imag - point.imag)**2) for point in constellation])


## Functions for the types of encodings

### m-PSK

In [3]:
# Computes the probability of error for symbol i with PSK encoding (P_e(i) == P_e here !)
def error_m_psk(i, m, energy_per_symbol, noise_var = 10**(-2.65)):
    inner_part = lambda theta: np.exp(-(np.sin(np.pi/m)**2)**2 / np.sin(theta) * (energy_per_symbol/(2*(noise_var**2))))
    return (1/np.pi) * sp.quad(inner_part, 0, np.pi - (np.pi / m))

# Performs the m-PSK encoding
def m_psk_encoder(energy_per_symbol, codeword, alphabet, constellation = [], d=0):
    k = alphabet.index(codeword)
    m = len(alphabet)
    return np.sqrt(energy_per_symbol) * np.exp(2j * np.pi * (k/m))

### QAM

In [4]:
# Computes the probability of error for symbol i with QAM encoding (P_e(i) == P_e here !)
def error_qam(m, d, noise_var = 10**(-2.65)):
    func = q_function(d/(2*noise_var))
    return 2*func - func**2

# Computes the PAM constellation for a given number of points and a distance for each of these points.
def pam(n, d):
    right_side = np.array([d*i + (d/2) for i in range(0, int(n/2))])
    left_side = -right_side
    return np.append(np.sort(left_side), right_side)


# Computes the whole m-QAM constellation, to be used to map each codeword to a point generated here
# WARNING: m MUST BE A POWER OF 2 HERE !!!
def m_qam(m, d, max_energy = (3/2)):
    if d > max_energy:
        raise OverflowError("ERROR: the distance you have set is too big for the project.")

    axis_len = int(np.sqrt(m))
    if axis_len % 2 != 0:
        axis_len = axis_len + 1

    diff_with_nearest_square = m - axis_len**2

    # if the difference is positive => we need to add some more points
    if diff_with_nearest_square > 0:
        real_axis_pam = pam(diff_with_nearest_square, d)
    else:
        real_axis_pam = pam(axis_len, d)
    if real_axis_pam.max() > max_energy:
        raise OverflowError("ERROR: You have some points that are outside the constrained square. Try to lower the distance or reduce the size of the constellation.")
    
    imaginary_axis_pam = pam(axis_len, d)[::-1]
    points_pairs = np.array([[complex(re, im) for re in real_axis_pam] for im in imaginary_axis_pam])

    # if the difference is negative => we need to remove some points from the constellation
    if diff_with_nearest_square < 0:
        nb_elts_to_delete_per_half_row = np.sqrt(-diff_with_nearest_square)/2
        nb_rows_to_modify = -diff_with_nearest_square / (2*nb_elts_to_delete_per_half_row)

        # First we set them to 0
        for j in range(0, int(nb_rows_to_modify/2)):
            for i in range(0, int(nb_elts_to_delete_per_half_row)):
                points_pairs[j, i] = 0
                points_pairs[j, len(points_pairs)-1-i] = 0
                points_pairs[len(points_pairs)-1-j, i] = 0
                points_pairs[len(points_pairs)-1-j, len(points_pairs)-1-i] = 0

    # We flatten the array to make it easier to map the codewords
    points_pairs = points_pairs.flatten()

    # Then, we remove all the points set to 0 (impossible to have if the dimension is a full square, so no worries)
    points_pairs = np.delete(points_pairs, np.argwhere(points_pairs == 0.0+0.0j))

    return points_pairs


# Performs the QAM encoding (in a linear fashion for the moment => might be re-worked)
def m_qam_encoder(codeword, constellation, alphabet, energy_per_symbol=0):
    try:
        k = alphabet.index(codeword)
        m = len(alphabet)
        return constellation[k]
    except OverflowError as ovferr:
        raise

## Encoder part

In [5]:
# Encodes a string into a sequence of complex numbers & adds dummy points in front of it, given an encoder, a constellation, an alphabet and a number of dummy points
def encode_string(raw_str, n_dummy_symbols, encoder, dummy_symbol, constellation = [], alphabet = []):
    if n_dummy_symbols >= 100:
        raise OverflowError("ERROR: The batch of dummy samples cannot be equal or exceed 100 symbols.") 
    bit_str = ascii_str_to_binary(raw_str)
    splitted_bit_str = split_bit_str(bit_str, int(np.log2(len(alphabet))))
    splitted_bit_str_size = len(splitted_bit_str)
    if splitted_bit_str_size > 100:
        raise OverflowError("ERROR: The string without the dummy symbols cannot exceed 100 symbols.") 

    if splitted_bit_str_size + n_dummy_symbols > 100:
        raise OverflowError(f"ERROR: the number of dummy samples is too high, the max value you can set here is {100 - splitted_bit_str_size}.")
            
    theta_estimator_batch = np.full((n_dummy_symbols, 1), dummy_symbol)

    return np.append(theta_estimator_batch, np.array([encoder(codeword=codeword, constellation=constellation, alphabet=alphabet) for codeword in splitted_bit_str]))

## Channel part (given)

In [6]:
def channel(sent_signal):
    s = np.mean(sent_signal**2)
    if s <= 1:
        s = 1
    noise_power = (10**(-2.65))*s
    shift = np.exp(-2j*np.pi*np.random.rand())
    print(f"Applied shift: {np.angle(shift)}")
    sent_signal = sent_signal*shift
    noise_std = np.sqrt(noise_power/2)
    rcv_signal = sent_signal + noise_std*np.random.randn(len(sent_signal)) + 1j*noise_std*np.random.randn(len(sent_signal))
    return rcv_signal

## Decoder part

In [7]:
def decode_str(noisy_str, dummy_symbol, n_dummy_symbols, alphabet, constellation):
    # Isolate the dummy symbols to estimate the phase shift
    dummy_symbols = noisy_str[0:n_dummy_symbols]
    
    # Estimate the phase shift (np.angle gives a value between (-pi, pi])
    phase = (np.angle(np.sum(dummy_symbols)) + np.pi) - (np.angle(dummy_symbol) + np.pi) 

    print(f"Estimated phase: {phase}, phase with normalization: {phase % 2*np.pi}")
    # Apply the inverse phase shift to every codeword
    dephased_symbols = np.exp(-1j * phase) * noisy_str[n_dummy_symbols:]

    decoded_codewords = [alphabet[minimum_distance_criterion(constellation, symbol)] for symbol in dephased_symbols]
    resulting_bit_str = "".join(decoded_codewords)
    return resulting_bit_str

## Putting everything together (for debugging)

In [8]:
str_to_process = "hellohowareyoutodayimtryingtodecodethestringliketheonegivenattheexambutitshard"

# 78-chars ASCII string
encoder = m_qam_encoder

original_bitstr = ascii_str_to_binary(str_to_process)
n_dummy_symbols = 9
max_energy = 3/2

dummy_symbol = max_energy + 1j * max_energy

print(f"Performing an encoding/decoding on the string {str_to_process}:")
for codeword_size in compute_codeword_lengths(ascii_str_to_binary(str_to_process)):
    ideal_distances = []
    for d in np.arange(0.15, max_energy, 0.001):
        try:
            # We limit the size of codewords to avoid unrealistic constellations given our max-energy constraint
            if 2 <= codeword_size and codeword_size <= 10:
                print(f"For a {2**codeword_size}-QAM encoder with distance d = {d}:")
                constellation = m_qam(2**codeword_size, d)
                alphabet = get_alphabet_from_codeword_length(codeword_size)

                # We encode the string
                encoded_str = encode_string(str_to_process, constellation=constellation, alphabet=alphabet, n_dummy_symbols=n_dummy_symbols, dummy_symbol=dummy_symbol, encoder=encoder)

                # We pass it to the channel
                noisy_result = channel(encoded_str)

                # We decode it finally
                print("Decoded bit-string:")
                decoded_bitstr = decode_str(noisy_result, dummy_symbol=dummy_symbol, n_dummy_symbols=n_dummy_symbols, alphabet=alphabet, constellation=constellation)
                diff_positions = get_diff_positions(original_bitstr, decoded_bitstr)
                print(f"Number of differences (in bits from the original string): {len(diff_positions)} ->", diff_positions)

                print("Decoded ASCII string:")
                print(binary_to_ascii_str(decoded_bitstr))

                if len(diff_positions) == 0:
                    ideal_distances.append(d)      
        except OverflowError as ovferr:
            print(ovferr)
            print("--------------------")
            break # Go to another QAM configuration when we reach/go past the max-energy limit
        except UnicodeDecodeError as unicodeerr:
            print(unicodeerr)
            print()
            continue

    print(f"The ideal distances for a {2**codeword_size}-QAM are: {ideal_distances}")   

Performing an encoding/decoding on the string hellohowareyoutodayimtryingtodecodethestringliketheonegivenattheexambutitshard:
The ideal distances for a 2-QAM are: []
For a 4-QAM encoder with distance d = 0.15:
ERROR: The string without the dummy symbols cannot exceed 100 symbols.
--------------------
The ideal distances for a 4-QAM are: []
For a 8-QAM encoder with distance d = 0.15:
ERROR: The string without the dummy symbols cannot exceed 100 symbols.
--------------------
The ideal distances for a 8-QAM are: []
For a 64-QAM encoder with distance d = 0.15:
Applied shift: -2.876116303948269
Decoded bit-string:
Estimated phase: -2.878617514549398, phase with normalization: 3.5229269781558745
Number of differences (in bits from the original string): 15 -> [12, 13, 14, 101, 134, 138, 139, 140, 243, 244, 245, 332, 534, 535, 536]
Decoded ASCII string:
                                                                                                                                              

## Using the provided server

### Functions to use the provided server

In [ ]:
def serialize_complex(complex_vector,file_name):
    complex_vector = complex_vector.reshape(-1)
    np.savetxt(file_name,np.concatenate([np.real(complex_vector),
    np.imag(complex_vector)]))

def deserialize_complex(file_name):
    tx_data = np.loadtxt(file_name)
    N_sample = tx_data.size
    N_sample = N_sample//2
    tx_data = tx_data[0:N_sample] + 1j*tx_data[N_sample:(2*N_sample)]
    return tx_data

### Setting up the hyper-parameters, constellation & alphabet

In [ ]:
# str_to_process = "hellohowareyoutodayimtryingtodecodethestringliketheonegivenattheexambutitshard" # 78-chars ASCII string
encoder = m_qam_encoder

original_bitstr = ascii_str_to_binary(str_to_process)
codeword_size = 8 # TODO: FIND BEST PARAMETER VALUE
d = 0.191 # TODO: FIND BEST PARAMETER VALUE
n_dummy_symbols = 22 # TODO: FIND BEST PARAMETER VALUE
max_energy = 3/2

# For a m-QAM constellation & associated 256-symbols alphabet
constellation = m_qam(2**codeword_size, d)
alphabet = get_alphabet_from_codeword_length(codeword_size)

### Encoding the string and serializing it

In [ ]:
encoded_str = encode_string(str_to_process, constellation=constellation, alphabet=alphabet, n_dummy_symbols=n_dummy_symbols, energy_per_symbol=1, encoder=encoder)
serialize_complex(encoded_str, "input.txt")

### De-serializing the file and decoding the string

In [ ]:
noisy_result = deserialize_complex("output.txt")
decoded_bitstr = decode_str(noisy_result, n_dummy_symbols=n_dummy_symbols, alphabet=alphabet, constellation=constellation)
print(get_diff_positions(original_bitstr, decoded_bitstr))
print(binary_to_ascii_str(decoded_bitstr))